In [1]:
import sys
import datetime

sys.path.append("..")

from utils.get_settings import parse

settings = parse("../utils")

settings

{'train': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv',
  'pickle_path': '../temp_result/train_data.pkl',
  'pickle_path_augument': '../temp_result/train_data_a.pkl'},
 'test': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv',
  'pickle_path': '../temp_result/test_data.pkl',
  'pickle_path_augument': '../temp_result/test_data_a.pkl'},
 'valid': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/valid_data.pkl',
  'pickle_path_augument': '../temp_result/valid_data_a.pkl'},
 'stage2': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/stage2_data.pkl'}}

In [3]:
import torch.nn as nn
class MLP(nn.Module):
    def __init__(self, bert_hidden_size):
        super().__init__()
        self.bert_hidden_size = bert_hidden_size
        self.fc = nn.Sequential(
            nn.Dropout(0.3),
            nn.Linear(bert_hidden_size, 1),

        )
        
    def forward(self, vector):
        return self.fc(vector)

In [4]:
from joblib import dump,load
model_name = "bert_large_rd0"
bert_model = load(model_name+".joblib")

In [5]:
bert,mlp = bert_model

In [10]:
import pandas as pd
test_data = pd.read_pickle(settings["stage2"]["pickle_path"])

In [12]:
import numpy as np
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch.nn as nn
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')

In [14]:
# form test_data
test_data['indexed_tokens'] = test_data['bert_tokens_torch'].apply(lambda x: tokenizer.convert_tokens_to_ids(x))
test_indexed_tokens = list(test_data['indexed_tokens'])
test_offsets_A = list(test_data['A_idx_bert_torch'])
test_offsets_B = list(test_data['B_idx_bert_torch'])
test_offsets_pron = list(test_data['pron_idx_bert_torch'])
test_offsets_other = list(test_data['neither_idx_bert_torch'])
test_offsets_cls = [np.array([0])]*len(test_offsets_other)
print('Number of testing data is:',len(test_data))

Number of testing data is: 454


In [15]:
def extract_vectors(bert_last_layer, offsets, offsets_mask):
    extracted_vectors = bert_last_layer.gather(
        1, offsets.unsqueeze(2).expand(-1, -1, 1))
    vector_masked = extracted_vectors.mul(offsets_mask.unsqueeze(2).float()).sum(1)/(offsets_mask.sum(1).unsqueeze(1).float())
    return vector_masked
  
def get_padded_and_mask(sequence, left, right):
    seq_ori = sequence[left:right]
    seq_padded = pad_sequence([torch.Tensor(v) for v in seq_ori],batch_first = True).cuda().long()
    mask = (seq_padded != 0).cuda().long()
    return seq_padded, mask

In [18]:
from torch.nn.utils.rnn import pad_sequence
re = []
bert.eval()
mlp.eval()
with torch.no_grad():
    for b in range(0,len(test_offsets_cls),1):
        start = b
        end = min(len(test_offsets_cls),b+1)
        tr_X_padded, attention_mask = get_padded_and_mask(test_indexed_tokens, start, end)
        tr_A_padded, tr_A_mask = get_padded_and_mask(test_offsets_A, start, end)
        tr_B_padded, tr_B_mask = get_padded_and_mask(test_offsets_B, start, end)
        tr_other_padded, tr_other_mask = get_padded_and_mask([np.array(list(i)+list(j)) for i,j in zip(test_offsets_cls,test_offsets_pron)], start, end)
        segments_tensors = torch.zeros(tr_X_padded.size()).cuda().long()

        encoded_bert, _ = bert(tr_X_padded, segments_tensors,attention_mask,output_all_encoded_layers=False)

        pred_ = mlp(encoded_bert)
        pred = torch.cat([extract_vectors(pred_, tr_A_padded, tr_A_mask),\
                                  extract_vectors(pred_, tr_B_padded, tr_B_mask),extract_vectors(pred_, tr_other_padded, tr_other_mask)],1)
        pred_data = pred.data.cpu().numpy()
        re.append(pred_data)


In [20]:
from scipy.special import softmax
pred = softmax(np.concatenate(re).reshape(-1,3),axis = 1)
dump(pred,"../ensemble/"+ model_name+'+stage2.joblib')

['../ensemble/bert_large_rd0+stage2.joblib']